# Présentation

Prudential est l'un des émetteurs d'assurance vie majeur aux Etats-Unis.<br>
Le processus d'acces a une assurance vie prend en moyenne 30 jours (planification d'examens médicaux, identification des risques, demande d'information), ce qui peut décourager un client voulait souscrire à une assurance.

Il n'y a donc que 40% des ménages américains qui possèdent une assurance vie.

Prudential veut raccourcir ce processus en conservant la confidentialité du client.
Pour ce faire, Prudential constuit un modèle prédictif qui classe les risques grâce à une approche automatisée

# Plan
<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Sommaire</h3>
    
* [Objectifs](#0)
    
<font size=+1><b>Chargement des données</b></font>
* [Import des fichiers](#1)
* [Import des librairies](#2)

<font size=+1><b>Visualisation des fichiers</b></font>
* [Fichier test](#3)
* [Fichier train](#4)
* [Description des données](#5)
* [Description des variables](#6)
* [Typage des variables](#7)
* [Visualisation avec pandas_profiling (trop gourmand)](#8)
    
<font size=+1><b>Analyse exploratoire</b></font>
* [Visualisation numérique des variables](#9)
    ** [Variable cible](#10)
    ** [Variables Product_Info](#11)
    ** [Variable âge](#12)
    ** [Variable taille](#13)
    ** [Variable poids](#14)
    ** [Variable IMC](#15)
    ** [Variables Employment_Info](#16)
    ** [Variables Insuranced_Info](#17)
    ** [Variables Insurance_History](#18)
    ** [Variables Family_Hist](#19)
    ** [Variables Medical_Keys](#20)
* [Correlation entre variables](#21)
* [Correlation des features avec la variable cible](#22)    
        
<font size=+1><b>Analyse des données manquantes</b></font>
* [Détection de données manquantes](#9)
* [Contenu des variables les moins renseignées](#10)
* [Valeurs 'null' ](#11)
* [Gestion des données manquantes](#12)

    
<font size=+1><b>Modèle Bayésien</b></font>
* [Explication du modèle](#13)
* [
    
    
    
    
    
  

## Feuille de route
### Fait:
* Analyse des fichiers
* Etude exploratoire 
* Incidence des features sur variable "response"

### A faire:
* Réduction dimensionnelle features correllées
* Réduction dimensionnelle features pas pertinentes
* Introduction modèle bayésien
* évaluation de la pertinence du modèle


<a id="0"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>

# Objectifs


Dans ce jeu de données on dispose de plus d'une centaine de variables décrivant les caractéristiques des demandeur d'assurance-vie.

Le but est de prédire avec le plus de certitude la variable <b>Response</b> pour chaque potentiel client (les données sont anonymisée et on parlera d'ID).

Cette variable <b>Response</b> est une mesure de risque de l'ID et qui comporte 8 valeurs classant la risque .

Les résultats ont pour but de rationaliser l'importance de certaines variables afin d'aider Prudential à mieux prédire les devis.

Dans ce Notebook, on fera tout d'abord une exploration des données fournies, puis nous tenterons une approche bayésienne afin de mesurer son efficacité.

# Chargement des données

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import des librairies nécéssaires et des données

<a id="1"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Import des fichiers


In [ ]:
# import des données
train = pd.read_csv('../input/prudential-life-insurance-assessment/train.csv.zip')
test = pd.read_csv('../input/prudential-life-insurance-assessment/test.csv.zip')

<a id="2"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Import des librairies

In [ ]:
# librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Visualisation des fichiers

<a id="3"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## fichier test

In [ ]:
# dimension du df et visualisation des données
print(test.shape)
test.head(3)

In [ ]:
# liste des variables du df test
list(test)

<a id="4"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## fichier train

In [ ]:
# dimension du df et visualisation des données
print(train.shape)
train.sample(3)

In [ ]:
# valeurs de la variable response
train.Response.unique()

La variable Response contient 8 valeurs différentes. Entiers de 1 à 8 compris.

In [ ]:
# liste des variables du df train
list(train)

<a id="5"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Description des données
Les deux jeux de données contiennent les mêmes variables -127 différentes- le fichier train contient la variable "Response" qui est la variable a prédire. Elle prend une valeur numérique entière comprise entre 1 et 8.
1. train.csv: le jeu d'entraînement contient pres de 60000 IDs et 127 variables en plus de la valeur "Response"
2. test.csv: le jeu de test dans lequel on doit prédire la variable "Response" pour chacun des 19765 IDs qu'il contient.

<a id="5"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Description et typage des variables:

In [ ]:
# description des variables 'train'
train.info(verbose=True)

In [ ]:
# description des variables 'test'
test.info(verbose=True)

Il n'y a que des variables numériques que ce soit des flottants (14), ou des entiers (108 ou 109 pour le jeu d'entraînement).
On a dans les deux jeux une variable objet associée à la variable ***'Product_Info_2'***

## Interprétation des variables:
* ***Id***: identifiant unique
* ***Product_Info_1 -> Product_Info_7***: Un ensemble de variables normalisées relatives à l'assurance à laquelle on veut souscrire
* ***Ins_Age***: Age de la personne
* ***Ht***: Taille de la personne
* ***Wt***: Poids de la personne
* ***BMI***: Indice de masse corporelle de la personne
* ***Employment_Info_1 -> Employment_Info_6***: Ensemble de variables informant de toutes les expériences de travail de la personne
* ***InsuredInfo_1 -> InsuredInfo_7***: Ensemble de variables compilant des informations sur la personne
* ***Insurance_History_1 -> Insurance_History_9***: Ensemble de variables relatives aux historiques d'assurance du demandeur
* ***Family_Hist_1 -> Family_Hist_5***: Ensemble de variables relatives aux antécédents familiaux de la personne
* ***Medical_History_1 -> Medical_History_41***: Ensemble de variables relatives aux antécédents médicaux de la personne
* ***Medical_Keyword_1 -> Medical_Keyword_48***: Ensemble de variables associées à la présence ou à l'absence d'un mot clé médical. Valeur Booléenne (0 indiquant l'absence, 1 la présence du terme médical)
* ***Response***: C'est la variable cible qu'on cherche à prédire. Elle est comprise entre 1 et 8 et est relative à la décision finale associée à une demande d'assurance faite par la personne

### On se penche sur la variable objet ***'Product_Info_2'*** 

In [ ]:
print("Il y a {} valeurs prises par la variable Product_Info_2 fichier de test: \n{}".format(len(test.Product_Info_2.unique()),test.Product_Info_2.unique()))
print("Il y a {} valeurs prises par la variable Product_Info_2 fichier d'entraînement: \n{}".format(len(train.Product_Info_2.unique()),train.Product_Info_2.unique()))

Product_Info_2 prend 19 valeurs différentes, que ce soit dans le fichier d'entraînement ou dans celui de test. Les valeurs qu'elle prend est une combinaison d'une lettre et d'un chiffre.
- A1 -> A7
- B1 -> B2
- C1 -> C4
- D1 -> D4
- E1

In [ ]:
# informations sur le jeu d'entrainement
train.info()

<a id="6"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Visualisation avec pandas_profiling (décommenter les cellules pour générer le rapport)

In [ ]:
# librairie pour Rapport de données

# pour visualisation des données avec la bibliothèque ProfileReport
# /!\ tres gourmand en cpu peut faire planter 
# sûrement à cause du grand nombre de variables 
from pandas_profiling import ProfileReport

In [ ]:
# aperçu des données d'entrainement
#train_profile = ProfileReport(train, title="Rapport sur les données d'entraînement", html={'style': {'full_width': True}}, sort="None")
#train_profile

In [ ]:
# aperçu des données de test
#test_profile = ProfileReport(test, title="Rapport fichier test", html={'style': {'full_width': True}}, sort="None")
#test_profile

## Visualisation de répartition des variables

### Variable cible

In [ ]:
# distribution de la variable cible
plt.figure(figsize=(12,6))
sns.countplot(train.Response).set_title('Distribution de la variable cible "Response"')
plt.grid(linestyle='dotted')
plt.show()

Observations:

Ce qu'on peut penser de la variable response: elle correspond à une catégorie de classement de risque médicaux pour une personne donnée<br>
On voit que la réponse donnée aux clients est le plus régulierement la numéro 8 avec pres de 20000 valeurs, suivent les valeurs 6 et 7 (resp. plus de 10000 et 7500). Il ya peu de  réponse 3 & 4<br>
On cherche a determiner lesquelles de ces valeurs représentent des réponses positives. Pour cela on imagine qu'une réponse dépend des conditions de santé du client et on présume que le poids est un facteur de réponse négative. On affiche le poids et la réponse en triant en fonction du poids

In [ ]:
# affichage response et poids trié 
train[['Wt', 'Response']].sort_values(by='Wt', ascending=False).tail(20)
train[['Wt', 'Response']].sort_values(by='Wt', ascending=False).head(20)

On dégage une tendance: <br>
la réponse 1 est tres souvent donnée pour les personne pesant le plus. On peut aisément imaginer qu'il s'agit de personnes en surpoids et donc qui comportent un potentiel risque médical.
On déduit donc que les valeurs de la variable 'Response' considèrent le dossier d'un potentiel souscripteur comme allant de 1: 'dossier défavorable/personne en tres mauvaise santé' à 8: 'dossier tres favorable/personne en excellente santé'

On définit les colonnes concernées par chacun des types d'informations du dataframe

In [ ]:
# definition des colonnes en fonction du typoe d'information qu'elles portent
Product_info_cols = ['Product_Info_{}'.format(i) for i in range(1,7)]
Insured_info_cols = ['InsuredInfo_{}'.format(i) for i in range(1,7)]
Insurance_hist_cols = ['Insurance_Hist_{}'.format(i) for i in range(1,9)]
Family_hist_cols = ['Family_Hist_{}'.format(i) for i in range(1,5)]
Medical_hist_cols = ['Medical_History_{}'.format(i) for i in range(1,41)]
Medical_key_cols = ['Medical_Keyword_{}'.format(i) for i in range(1,48)]

# regroupe les infos de chaque variables
Product_info_data = pd.concat([train[Product_info_cols], test[Product_info_cols]])
Insured_info_data = pd.concat([train[Insured_info_cols], test[Insured_info_cols]])
Insurance_hist_data = pd.concat([train[Insurance_hist_cols], test[Insurance_hist_cols]])
Family_hist_data = pd.concat([train[Family_hist_cols], test[Family_hist_cols]])
Medical_hist_data = pd.concat([train[Medical_hist_cols], test[Medical_hist_cols]])
Medical_key_data = pd.concat([train[Medical_key_cols], test[Medical_key_cols]])



In [ ]:
# fonction de tracé des distributions
def plots_cols(data):
    nb_cols = len(data.columns)
    fig = plt.figsize(6*6,6(nb_cols//6+1))
    for i, col in enumerate(data.columns):
        cpt = Counter(data[col])
        keys = list(cpt.keys())
        vals= list(cpt.values())
        plt.subplot(nb_cols//6+1,6,i+1)
        plt.bar(range(len(keys)), vals, align='center')
        plt.xticks(range(len(keys)), keys)
        plt.xlabel(col)
        plt.ylabel('Distribution')
        plt.gird(linestyle='dotted')
        plt.show()

### Variables product info

In [ ]:
plots_cols(Product_info_cols)

### Variables personne

In [ ]:
#fusion de test/train et appercu Id
all_data = train.append(test).sort_values(by='Id')
print(len(all_data.Id.unique()))
all_data.shape

In [ ]:
all_data

In [ ]:
# age
plt.figure(figsize=(12,6))
sns.distplot(all_data['Ins_Age']).set_title('Distribution âge')
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# taille
plt.figure(figsize=(15,6))
sns.distplot(train['Ht']).set_title('Distribution taille')
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# poids
plt.figure(figsize=(12,6))
sns.distplot(train['Wt']).set_title('Distribution poids')
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# Indice de masse corporelle
plt.figure(figsize=(12,6))
sns.distplot(train['BMI']).set_title('Distribution de la masse corporelle')
plt.grid(linestyle='dotted')
plt.show()

### Variables Employment_info

In [ ]:
# Employment_info

### Variables Insuranced_info

In [ ]:
# Insuranced_info

### Variables historique d'assurance

In [ ]:
#Insurance_History

### Variables antécédants familiaux

In [ ]:
# Family_Hist

### Variables mots clés médicaux

## Correlation entre variables

In [ ]:
# matrice de correlation
correlation = train.corr()

# matrice triangle
mask = np.zeros_like(correlation, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# configuration de la matrice
f, ax = plt.subplots(figsize=(30,30))

# affichage
sns.heatmap(correlation, cmap='seismic', mask=mask, vmax=1, vmin=-1, center=0, square=False)
plt.show()

In [ ]:
# fonction vecteur de corrélation
def correlation_vector(df, feature):
    corr_matrix = df.corr()
    corr_vector = corr_matrix[feature]
    corr_vector = corr_vector.sort_values(ascending=False)
    corr_vector = corr_vector.drop(feature)
    corr_vector =pd.DataFrame(corr_vector)
    f, ax = plt.subplots(figsize=(3,30))
    plt.title('Vecteur de corrélation variable ' + feature)
    sns.heatmap(corr_vector, cmap='seismic', vmax=1, vmin=-1, center=0, linewidth=0.5, cbar_kws={'shrink': .5}, annot=True, fmt='1.3f', cbar=False)

    
# Affichage corrélation entre la variables Response et les autres variables
correlation_vector(train, 'Response')

## Analyse des données manquantes

In [ ]:
def missing(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Pourcent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return tt

In [ ]:
# verification des données manquantes dans le jeu d'entrainement
missing(train)['Pourcent'].sort_values(ascending=False)

In [ ]:
# affichage des 10 variables de 'train' pour lesquelles il manque le plus d'information
missing(train)['Pourcent'].sort_values(ascending=False).head(10)

In [ ]:
# verification des données manquantes dans le jeu de test
missing(test)['Pourcent'].sort_values(ascending=False)

In [ ]:
# affichage des 10 variables de 'test' pour lesquelles il manque le plus d'information
missing(test)['Pourcent'].sort_values(ascending=False).head(10)

On choisit de ne pas orendre en compte les variables pour lesquelles il manque plus de 75% des données

In [ ]:
train_modified = train[train.columns[train.isnull().mean() <= 0.75]]
test_modified = test[test.columns[test.isnull().mean() <= 0.75]]

In [ ]:
# affichage des valeurs pour les données d'entrainement
train_modified.isnull().sum().sort_values(ascending=False)

In [ ]:
# On regarde dans quelles variables il manque des informations dans train
train_modified.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
# affichage des valeurs pour les données de test
test_modified.isnull().sum().sort_values(ascending=False)

In [ ]:
# On regarde dans quelles variables il manque des informations dans test
test.isnull().sum().sort_values(ascending=False).head(10)

### Observations:
On constate qu'après avoir droppé les variables dont il manquait plus de 75% des données dans les fichier train et test.
Il reste 9 variables pour lesquelles il manque des informations que ce soit pour le jeu de test ou pour celui d'entraînement. Remarquons que ce sont les mêmes variables qui sont concernées.

### Observation des valeurs 'null'

In [ ]:
# on liste les colonnes dans lesquelles on a des valeurs égale à 'null'
list_null_train = train.columns[train.isna().any()].tolist()
list_null_test = test.columns[test.isna().any()].tolist()

# affichage des variables concernées
print(list_null_test)
print(list_null_train)


In [ ]:
train.info(verbose=True)